<a href="https://colab.research.google.com/github/Dagger77/amzn_recsys/blob/main/Velotix_Amazon_Reviews_2018_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/Velotix HA/archive.zip' -d './amazon_data/'

Archive:  /content/drive/MyDrive/Velotix HA/archive.zip
  inflating: ./amazon_data/amazon_reviews.csv  


##IMPORT

In [ ]:
import os

import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import html
from bs4 import BeautifulSoup
import ast

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

##FUNCTIONS

In [ ]:
def clean_html(text):
    # clean text from pieces of html code
    if pd.notnull(text):
        soup = BeautifulSoup(text, 'html.parser')
        return soup.get_text(separator=" ")
    return text

In [ ]:
def clean_list_as_string_feature(list_as_string):
    # convert text feature presented as str(list) to text
    try:
        list_representation = ast.literal_eval(list_as_string)
        cleaned_list = [item.strip().rstrip('.') for item in list_representation if item.strip()]
        result = '. '.join(cleaned_list) + '.' if cleaned_list else ''
        return result if result else 'No record'
    except (ValueError, SyntaxError):
        return list_as_string

In [ ]:
def clean_other(text):
    #Clean specific cases that can't be cleaned by BS and html.unescape
    to_clean = ['\n', '\\n', '&Quot', '&Amp', '<br>']
    for chars in to_clean:
        text = text.replace(chars, '')
    return text.strip()

##CLEANING

In [ ]:
df = pd.read_csv('/content/amazon_data/amazon_reviews.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551159 entries, 0 to 551158
Data columns (total 14 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   userName     551026 non-null  object 
 1   verified     551159 non-null  bool   
 2   itemName     551158 non-null  object 
 3   description  551151 non-null  object 
 4   image        551159 non-null  object 
 5   brand        547287 non-null  object 
 6   feature      551159 non-null  object 
 7   category     551159 non-null  object 
 8   price        476825 non-null  object 
 9   rating       551159 non-null  float64
 10  reviewTime   551159 non-null  object 
 11  summary      550916 non-null  object 
 12  reviewText   550731 non-null  object 
 13  vote         551159 non-null  int64  
dtypes: bool(1), float64(1), int64(1), object(11)
memory usage: 55.2+ MB


In [ ]:
df.head()

,userName,verified,itemName,description,image,brand,feature,category,price,rating,reviewTime,summary,reviewText,vote
0,Amazon Customer,True,"Toblerone Swiss Milk Chocolate Bar, Crunchy Salted Almond, 3.52 Ounce","[""Made from deliciously decadent ingredients, Toblerone's swiss milk chocolate is a delight for any chocolate lover. This special bar is packed with crunchy salted caramelized almonds along with T...","['https://images-na.ssl-images-amazon.com/images/I/51ncjOQJQ0L._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/518ZZLIcztL._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/imag...",Toblerone,[],Prime_Pantry,$1.63,5.0,2018-01-01,Five Stars,super smooth and yummy with crunchy bits,0
1,Carla,True,"Prince of Peace Organic Tea, Oolong, 100 Tea Bags","['Prince of Peace Enterprises, Inc., founded in 1983, earned its reputation as a leading and diversified company, specializing in the marketing, importing and distribution of gourmet products, bev...","['https://images-na.ssl-images-amazon.com/images/I/61ZBUFMoWML._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/4159d8l8Q5L._SS40_.jpg']",Prince Of Peace,[],Grocery_and_Gourmet_Food,$6.40,5.0,2018-01-01,Five Stars,Perfect for kombucha,0
2,Jaclyn,True,"Pet Champion Adjustable No-Pull Harness, Collar, Leash Matching Bundle Small, Medium, Large Dog","[""The Pet Champion Large/ Extra Large 22-36 inch Chest Bright Polyester Step-In Halter Dog Harness, Pink Lemonade offers more comfort and less tug while walking. You will have no problem walking t...","['https://images-na.ssl-images-amazon.com/images/I/41x6wIUiAlL._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/415zFmWBqkL._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/imag...",Pet Champion,"['Features Bright Pink Lemonade pattern', 'Dual D-ring style leash attachment for less tug', 'Made from 1 inch wide 100% high-grade polyester', 'Step-In Halter design for easier aplication', 'Larg...",Pet_Supplies,$7.99,5.0,2018-01-01,she has an odd shape chest and her pull over harness was not going over her big head so easy anymore. This harness finally adjusted to her size ...,Finally a harness that fits my puppy. I really was in need of a harness that fits my 7 pound pug Chihuahua mix.she has an odd shape chest and her pull over harness was not going over her big head ...,0
3,Christinne M,False,"Koh-I-Noor Progresso Woodless Colored 24-Pencil Set, Assorted Colored Pencils (FA8758.24)","['Koh-I-Noor Progresso Woodless Colored 24-Pencil Set, Assorted Colors contains rich, pigmented, solid color encased in lacquer. Five times the color as wood-cased pencils, these woodless pencils ...",['https://images-na.ssl-images-amazon.com/images/I/4165rMdS1RL._SS40_.jpg'],KOH-I-NOOR,"['Rich, pigmented, solid color encased in lacquer', 'Five times the color as wood-cased pencils', 'Excellent laydown properties for layering and blending without waxy buildup', 'Create dense, even...",Arts_Crafts_and_Sewing,$14.18,5.0,2018-01-01,Loving!!,"I LOVE THEM!! I bought them at Micheals our of curiosity and wanting to try a different colored pencil. I own Crayola, Prisma and Faber Castell. These colored pencils are very pigmented and there ...",0
4,Sandy,True,"Parker Urban Premium Ebony Metal Chiseled, Ballpoint Pen with Medium Black refill (S0911510)","[""Stunning craftsmanship based on a classic style established in 1921. Ebony black with metallic highlights and a gloss varnish. Enhanced with an asymmetric chiselled pattern that echos urban's er...","['https://images-na.ssl-images-amazon.com/images/I/31Fk3or2lDL._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/41tI1d6s3fL._SS36_.jpg', 'https://images-na.ssl-images-amazon.com/imag...",Parker,"['Ebony black with metallic highlights and a gloss varnish', ""Enhanced with an asymmetric chiseled pattern that echo's Urban's ergonomic form"", 'Nib in stainless steel for superior resistance and ...",Office_Products,NaN,5.0,2018-01-01,I love this pen,"I love th

In [ ]:
# It looks like dataset has a lot of duplicates

# dulpicates = df[df.duplicated(keep=False)]
# dulpicates.to_csv('amazon_duplicates.csv', index=False)

In [ ]:
print(f'Number of duplicates: {len(df[df.duplicated(keep=False)])}')

Number of duplicates: 80428


In [ ]:
df = df.drop_duplicates()

In [ ]:
df.info()
#510363 users not null
#510489 item not null

<class 'pandas.core.frame.DataFrame'>
Index: 510490 entries, 0 to 551158
Data columns (total 14 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   userName     510363 non-null  object 
 1   verified     510490 non-null  bool   
 2   itemName     510489 non-null  object 
 3   description  510482 non-null  object 
 4   image        510490 non-null  object 
 5   brand        507010 non-null  object 
 6   feature      510490 non-null  object 
 7   category     510490 non-null  object 
 8   price        440864 non-null  object 
 9   rating       510490 non-null  float64
 10  reviewTime   510490 non-null  object 
 11  summary      510253 non-null  object 
 12  reviewText   510089 non-null  object 
 13  vote         510490 non-null  int64  
dtypes: bool(1), float64(1), int64(1), object(11)
memory usage: 55.0+ MB


###userName & itemName

In [ ]:
# Since we are building recommender system, User and Item entities are crucial.
# NaN must be dropped
df = df.dropna(subset=['userName', 'itemName'])

In [ ]:
#Some entries have HTML entities like &amp, &quot, etc. Some even have nested HTML entities
#For nested cases let's clean entries several times
for i in range(5):
  df.loc[:, 'userName'] = df['userName'].apply(html.unescape)
  df.loc[:, 'itemName'] = df['itemName'].apply(html.unescape)

In [ ]:
#Found some items having corrupted itemName having chunks of parcing code errors
bad_value_pattern = r'var aPageStart'
df.loc[df['itemName'].str.contains(bad_value_pattern, na=False), 'itemName'] = 'Nike Womens Flex Supreme TR 4 training shoe'

In [ ]:
df[800:820][['userName', 'itemName']]

,userName,itemName
918,kim,"Laube Blade Case, Mini"
919,verified purchase,PetNC Natural Care Hip & Joint Soft Chews
920,Amazon Dude,"Red Vines Sugar Free Licorice, Strawberry Flavor, 5oz Bags (12 Pack), Soft & Chewy Candy Twists"
921,Glenda Hunt,Catit Fountain
922,Pickle,Kohler 25-032-06-S Lawn & Garden Equipment Engine Oil Seal Genuine Original Equipment Manufacturer (OEM) Part
923,T. Shampo,"WESTERN 78057 Premium BBQ Products Post Oak Cooking Chunks, 570 cu inch Food"
924,M. L. Gaumer,Comfort Zone Multicat Refills
925,Got Influence?,"Incipio offGRID Samsung Galaxy S7 Battery Case, 3700 mAh Wireless Qi Charging Extended Battery Backup for Galaxy S7 - Black"
926,Music Man,"Oregon Scientific THGR810 10-Channel Wireless Remote Thermometer/Humidity Sensor with 10 Channels for the WMR200, WMR100, and WMR90 Weather Stations"
927,Lucian A. Sperta,"Prevue Pet Products Premium/Deluxe Cat Home, Black"


###description

In [ ]:
df.description.fillna('No description', inplace=True)

In [ ]:
#Clean specific cases that can't be cleaned by BS and html.unescape
df.loc[:, 'description'] = df['description'].apply(clean_other)

In [ ]:
#HTML code cleaning and HTML entities cleaning
df.loc[:, 'description'] = df['description'].apply(clean_html)
for i in range(5):
  df.loc[:, 'description'] = df['description'].apply(html.unescape)

<ipython-input-7-1f44697f75f0>:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


In [ ]:
#Final feature converting to text
df.loc[:, 'description'] = df['description'].apply(clean_list_as_string_feature)

In [ ]:
df[480:500].description

,description
568,"Nothing beats the tremendously tangy, playfully chewy chew of Airheads candy! This box is the delicious answer to your sweet tooth's dream and your bulk candy needs! Gluten-free, peanut-free and k..."
569,"Worry less while you're working, adventuring and living when you defend your device against drops, dirt and scrapes. Combining a solid internal shell with a resilient outer slipcover, Defender Ser..."
570,"Multipet Plush Dog Toy, Sock Monkey. Dogs like socks--as anyone who's returned home to find their laundry in shreds can attest. But giving your pooch a sock to play with just reinforces bad habits..."
571,Animal Crossing Series.
573,Tiny Footprint Coffee. Tiny Footprint Coffee Mindo Cloudforest Foundation Partnership (MCF). Statements regarding dietary supplements have not been evaluated by the FDA and are not intended to di...
575,Redline K9 Tug Toys are Made with high quality Bite Suit Material. This 3 x 10 Bite Suit Tug Toy is our most popular tug toy. Tug Toys are meant for interactive tugging between you and your dog on...
576,"Organic, Gluten Free, Vegan, quick cooking gravies. Great over nooldes, potatoes. Organic Brown Rice Flour, Organic Tapioca Starch, Organic Tamari Powder, Unrefined Salt, Organic Onion Powder, Org..."
577,"Rachael Ray Nutrish Real Salmon & Brown Rice Recipe is made with simple, natural ingredients, like real salmon, which is always the number one ingredient, combined with wholesome brown rice, veget..."
580,An odor free home starts with ARM & HAMMER Super Scoop Clumping Litter with Ammonia Odor Block formula. It combines powerful baking soda crystals with ammonia odor neutralizers to eliminate the to...
581,"Size:3XL-9XL 3XL: back length:45cm/17.71'', Chest:60cm/23.62'', Neck:40cm/15.74''. 4XL: back length:50cm/19.68'', Chest:65cm/25.6'', Neck:43cm/16.93''. 5XL: back length:55cm/21.65'', Chest:70cm..."


###brand

In [ ]:
df.loc[:, 'brand'] = df['brand'].fillna("No Name")

In [ ]:
#brand has some HTML entities, white spaces and new line symbols
df.loc[:, 'brand'] = df['brand'].str.replace('\n', '', regex=False).str.strip()
df.loc[:, 'brand'] = df['brand'].str.replace(r'\s+', ' ', regex=True).str.strip()
for i in range(5):
  df.loc[:, 'brand'] = df['brand'].apply(html.unescape)

###feature

In [ ]:
df.loc[:, 'feature'] = df['feature'].apply(clean_other)
df.loc[:, 'feature'] = df['feature'].apply(clean_html)
for i in range(5):
  df.loc[:, 'feature'] = df['feature'].apply(html.unescape)

<ipython-input-7-1f44697f75f0>:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


In [ ]:
df.loc[:, 'feature'] = df['feature'].apply(clean_list_as_string_feature)

In [ ]:
df[480:500].feature

,feature
568,No record
569,Compatible with Samsung Galaxy Note8. New SCREENLESS design: accommodates curved touchscreen. Try the OtterBox Alpha Glass screen protector for resistance against scratches and shattering (SKU: 77...
570,Sock Pal. Monkey. Plush. Dog Toy. Pets.
571,"Introducing amiibo: character figures designed to connect and interact with compatible games. By tapping the amiibo over your Wii U GamePad, you'll open up new experiences within each correspondin..."
573,No record
575,"Great reward toy for all breeds of dogs. Perfect for IPO, Schutzhund, Ring Sport, Police dogs and more. Made with RedLine K9's High Quality French Linen."
576,No record
577,"Real Salmon is the #1 Ingredient. No ground corn, wheat or soy ingredients. No poultry by-product meal or filler ingredients. No Artificial Flavors or Artificial Preservatives. Natural Cat Food wi..."
580,Great for fragrance sensitive cats and their owners. Combines powerful baking soda crystals with ammonia odor neutralizers to eliminate the toughest odors on contact. Rock-solid clumps let you rem...
581,"Measure Your Pet Read the Size Chart to Choose the Correct Size.If the measurements are 'between' two sizes, choose the larger size.(PS:According to our customers reviews,you can order 1 size up)..."


###price

In [ ]:
# Replace values that don't start with '$' with NaN
df.loc[df['price'].notna() & ~df['price'].str.startswith('$', na=False), 'price'] = np.nan

df.loc[df['price'].notna(), 'price'] = (
    df['price']
    .str.extract(r'(\$\d+\.\d{2})')[0]   # Extract the price in the format $xx.xx
    .replace('[\$,]', '', regex=True)    # Remove the $ sign
    .astype(float)                       # Convert to float
)
df['price'].fillna(df['price'].median(), inplace=True)

In [ ]:
print(df.price.min())
print(df.price.max())

0.0
999.99


###reviewTime

In [ ]:
df['reviewTime'] = pd.to_datetime(df['reviewTime'], format='%Y-%m-%d')

###summary      

In [ ]:
df.summary.fillna('No summary', inplace=True)

###reviewText

In [ ]:
df.reviewText.fillna('No review', inplace=True)

In [ ]:
df.loc[:, 'reviewText'] = df['reviewText'].apply(clean_other)
df.loc[:, 'reviewText'] = df['reviewText'].apply(clean_html)
for i in range(5):
  df.loc[:, 'reviewText'] = df['reviewText'].apply(html.unescape)

<ipython-input-7-1f44697f75f0>:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')
<ipython-input-7-1f44697f75f0>:4: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


In [ ]:
print(f'Number of duplicates: {len(df[df.duplicated(keep=False)])}')

Number of duplicates: 99


In [ ]:
df = df.drop_duplicates()

###image

In [ ]:
#for the sake of limited time, let's set aside image feature
df.drop(columns=['image'], inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 510312 entries, 0 to 551158
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   userName     510312 non-null  object        
 1   verified     510312 non-null  bool          
 2   itemName     510312 non-null  object        
 3   description  510312 non-null  object        
 4   brand        510312 non-null  object        
 5   feature      510312 non-null  object        
 6   category     510312 non-null  object        
 7   price        510312 non-null  float64       
 8   rating       510312 non-null  float64       
 9   reviewTime   510312 non-null  datetime64[ns]
 10  summary      510312 non-null  object        
 11  reviewText   510312 non-null  object        
 12  vote         510312 non-null  int64         
dtypes: bool(1), datetime64[ns](1), float64(2), int64(1), object(8)
memory usage: 51.1+ MB


##SAVE DATASET

In [ ]:
path = '/content/drive/MyDrive/Velotix HA/'
file_path = os.path.join(path, 'amazon_review_clean.parquet')

df.to_parquet(file_path)